# Classes and OOP in Python - described by example
##### Author: Everett Wetchler (everett.wetchler@gmail.com) 2019-10-20
---
EVERY variable in python has a "type," indicating what sort of thing it _is_ (integer, string, etc). **"Type" is effectively synonymous with "class."** To say that "x is an integer" is identical to saying that "x has class integer" or "x has type integer."

The class (type) determines:
1. What functions you can call on it. E.g. if `x` is a string, you can call `x.upper()` to get an uppercase version of it, but if `x` is an integer, running `x.upper()` will throw an error. 
2. How operators work on it. `1 + 2` returns `3` but `'1' + '2'` returns `'12'`, because `1` and `2` have class `int` while `'1'` and `'2'` have class `str`.
3. How you create one.
  * Some classes are defined in the core python language (integers, strings, lists, etc).  These are called _built-in types_, and each has a special syntax for creating it. For example `x = 1` makes x an integer, while `x = '1'` makes it a string, and `x = [1]` makes it a list (with one element, the integer 1).
  * For all other types (anything you define yourself, or `import`), you create them by calling the name and passing in any arguments needed to customize it. E.g.:

In [3]:
from pandas import Series

x = Series()
# or
y = Series([1,2,3], index=['a', 'b', 'c'])

### Important terminology

When you write `x = 1`, x is called an "instance" of the class integer. Put differently, a "class" is just a definition of some type-of-thing and how it works, while an "instance" is a single, actual thing in the flesh.

This also helps explain a useful python function, `isinstance`, which tells you if a variable has a certain class.

In [4]:
x = 4
print(isinstance(x, int))
print(isinstance(x, str))

True
False


Be sure not to confuse classes for instances.

In [5]:
isinstance(int, int)

False

## Defining your own classes, and why it's helpful

### Code encapsulation

Say you want to represent a user for an app. You could make a class for a user with a _class definition_ like so

In [6]:
class User:
    pass  # because you need *something* inside indented blocks in python, "pass" does nothing

Now, this is a profoundly boring class that does nothing. But it's valid! To create an instance of your class, do like so:

In [7]:
joe = User()

What if you want to give users a name, email, and password? Well, you could do it like this:

In [8]:
joe = User()
joe.name = "Joe"
joe.email = "joe@gmail.com"
joe.password = "wiggles123"

What's wrong with this? It works, but....
1. It's verbose. Wouldn't something like `joe = User("Joe", "joe@gmail.com", "wiggles123")` be nicer?
1. This isn't really any better than storing all this information in a dictionary. What are we gaining?
1. Anytime you create a user, you have to remember to set the name and email and password (and not make any typos!) Imagine if you had code elsewhere that (say) printed the email, e.g.

In [9]:
def print_email(some_user):
    print(some_user.email)

If you forgot to set the email (or accidentally mistype `email`)...

In [10]:
alice = User()
alice.name = 'Alice'
alice.password = 'wonderland'
alice.eemail = 'mad@hatter.com'

This code is gonna crash -- but not until you call it! Witness...

In [11]:
print_email(alice)

AttributeError: 'User' object has no attribute 'email'

Wouldn't it be nice to _guarantee_ that the name and email are set **when you create the user**?

#### Constructors

Classes let you control how instances are created ("constructed") by letting you define a special function called a _constructor_. It is a function you define _inside_ the class definition. It looks like this (pardon the confusing syntax, bear with me):

In [13]:
class User:
    def __init__(self, name, email, password):
        self.name = name
        self.email = email
        self.password = password

If you do this, you can then create a user like we discussed:

In [17]:
joe = User("Joe", "joe@gmail.com", "wiggles123")
print_email(joe)

joe@gmail.com


Why does this work? Well first, we defined a function on the class itself (we'll cover this more in a minute) -- but we gave it a special name, `__init__`. The first argument to `__init___` is always `self`, and it refers, in the function body, to the instance being created. The other arguments are simply parameters to the function, which you use to set attributes on the instance.

Because this is a special function name, Python knows that whatever you put in `__init__` translates into a constructor invoked on the name on the class with `MyClassName()`. `self` is also implicitly returned, so when you run `joe = User()` an instance is returned and set to the variable `joe`.

#### But wait, there's more!

Our constructors can do anything regular python code can do, so we can verify that everything was kosher in other ways. For example, we can check if the `email` looks valid -- say, by checking for an `@` symbol. If it's invalid, we can throw an error, so the program knows right away. Managing errors (called `Exception`s) in python is beyond our scope here, but this gives you the idea of using constructors to validate that everything looks okay.

In [20]:
class User:
    def __init__(self, name, email, password):
        self.name = name
        self.email = email
        self.password = password
        
        if '@' not in email:
            raise Exception("Invalid email:", email)

joe = User("Joe", "BAD EMAIL", "wiggles123")

Exception: ('Invalid email:', 'BAD EMAIL')

#### Custom functions

Often there are specific behaviors you may want to perform that are unique to instances of your class. For example,

In [10]:
class User:
    def __init__(self, name, email, password):
        self.name = name
        self.email = email
        self.password = password
    def say_hi(self):
        print("Hello", self.name)

joe = User("Joe", "joe@gmail.com", "wiggles123")
joe.say_hi()

Hello Joe


Note that you _cannot_ call `say_hi` on instances of any other type. E.g.
```python
x = 4
x.say_hi()
```
...would throw an error. This is because we have _encapsulated_ this code quite neatly, meaning `say_hi` only exists on this class.

Moreover, another class (say, an Admin class) could have its own function called `say_hi` and it would not interfere with User's `say_hi` because they live in different scopes. This lets us write arbitrarily complicated functionality for our objects and keep it neatly confined.

## Inheritance

All classes in python fit into a neat "family tree," where each class is a descendant of one (or more) other classes, going back all the way to a single common ancestor of everything: `object`

If you have an instance, say `x`, it is technically an instance of its class _and all of that class's ancestors_. For example, "basketball" is an instance of a "sport." But "sport" is a type of "game," so basketball is an instance of a game as well. And games are all types of "activity," so basketball is, at once, an instance of a sport, a game, and an activity.

And an `object`. Eventually, everything inherits from `object`. In other languages, they distinguish between _primitives_ (integers, strings, etc) and _objects_ (which have a class), but in python **everything** has a class, hence everything is an object.

From this, you can maybe guess that `isinstance(x, object)` will literally always return true. Everything is an object, somewhere in its ancestry!

In [11]:
x = 4
print(x, "is an int?", isinstance(x, int))
print(x, "is a string?", isinstance(x, str))
print(x, "is an object?", isinstance(x, object))
print()

y = "foo"
print(y, "is an int?", isinstance(y, int))
print(y, "is a string?", isinstance(y, str))
print(y, "is an object?", isinstance(y, object))
print()

4 is an int? True
4 is a string? False
4 is an object? True

foo is an int? False
foo is a string? True
foo is an object? True



### `issubclass`
While you can apply `isinstance` to an _instance_ of a class, you can apply `issubclass` to the _class_ itself. Note that classes are considered to be subclasses of themselves (see that `issubclass(int, int)` below returns True).

In [12]:
print("int is a subclass of str?", issubclass(int, str))
print("int is a subclass of int?", issubclass(int, int))
print("int is a subclass of object?", issubclass(int, object))
print("str is a subclass of object?", issubclass(str, object))
print("object is a subclass of str?", issubclass(object, str))

int is a subclass of str? False
int is a subclass of int? True
int is a subclass of object? True
str is a subclass of object? True
object is a subclass of str? False


In [13]:
issubclass(User, object)

True

### Inheritance makes things shine

To define a class (Foo) that inherits from another class (Bar), do this:

In [14]:
class Bar:
    pass

class Foo(Bar):
    pass

x = Foo()
print(isinstance(x, Bar))
print(issubclass(Foo, Bar))

True
True


That is, Foo _inherits from_ Bar. The beauty of this is that subclasses, by default, get alll the functionality from their parents:

In [15]:
class Bar:
    def simon_says(self, words):
        print('Simon says, "%s"' % words)

class Foo(Bar):
    pass

y = Bar()
y.simon_says("I am 'y'")

x = Foo()
x.simon_says("I am 'x'")

Simon says, "I am 'y'"
Simon says, "I am 'x'"


See how x, an instance of the Foo class, got to use the simon_says function without defining it? That's because Foo is a subclass of Bar, where simon_says is defined.

This makes us ready for the typical way that software engineering works - by building hierarchies of classes to share and encapsulate functionality.

Important: You can change the behavior defined by an ancestor. In this example, we can have the subclass (Foo) _override_ the behavior of its parent class (Bar) -- simply re-define the function in the subclass like so:

In [16]:
class Bar:
    def simon_says(self, words):
        print('Simon says, "%s"' % words)

class Foo(Bar):
    def simon_says(self, words):
        print('Simon likes to say, "%s"' % words)

y = Bar()
y.simon_says("I am 'y'")

x = Foo()
x.simon_says("I am 'x'")

Simon says, "I am 'y'"
Simon likes to say, "I am 'x'"


This is why, in oTree, many functions you can optionally specify. If you don't specify them, your class will inherit the default behavior from its parent.

### `super`

Sometimes you want to override an ancestor's behavior, but not lose the ability to do what the ancestor did. Let's say we want to define two classes: "Bar" and "BarBarBar" where both can `simon_says`, but `BarBarBar` simply does it three times. Here's how to do it, without duplicating code:

In [17]:
class Bar:
    def simon_says(self, words):
        print('Simon says, "%s"' % words)

class BarBarBar(Bar):
    def simon_says(self, words):
        for i in range(3):
            super().simon_says(words)

y = Bar()
y.simon_says("I am 'y'")

Simon says, "I am 'y'"


In [18]:
x = BarBarBar()
x.simon_says("I am 'x'")

Simon says, "I am 'x'"
Simon says, "I am 'x'"
Simon says, "I am 'x'"


## That's it for now! See if you can understand the oTree code better.